In [23]:
import re
import os
import json
from datetime import datetime
from typing import Optional

## text

In [24]:
# --- Funciones de procesamiento de texto (ya las tienes) ---

def fix_ocr_issues(text: str) -> str:
    correcciones = {
        "NACiÓN": "NACIÓN",
        "Colr,mbiana": "Colombiana",
        "Perscnal": "Personal",
        "AérEa": "Aérea",
        "No1956": "No 1956",
        "No.1956": "No. 1956",
        "N°1956": "N° 1956",
        "LEYNo": "LEY No",
        "LEYNo.": "LEY No.",
        "4JUN2019": "4 JUN 2019",
        "Dadaen": "Dada en",
        "Nc.": "No.",  # Agregamos esta corrección
        "2374'": "2374" #Agregamos esta correccion.
    }
    for error, correccion in correcciones.items():
        text = text.replace(error, correccion)

    # Mantener saltos de línea DOBLES, pero eliminar los sencillos DENTRO de párrafos.
    text = re.sub(r"(?<=\S)\n(?=\S)", " ", text)  # Junta líneas CON texto
    text = re.sub(r"\n{2,}", "\n\n", text)        # Máximo 2 saltos seguidos
    text = re.sub(r"[ \t]+", " ", text)  # Espacios/tabs a 1 espacio
    text = text.strip()
    text = re.sub(r"(No|N°|No\.)(\d+)", r"\1 \2", text, flags=re.IGNORECASE)
    text = re.sub(r"(\d+)([A-Z]{1,3}\d{4})", r"\1 \2", text)
    text = re.sub(
        r"(\d{4})\s+No\s+(\d{1,2}[A-Z]{3})",
        r"\2\1",
        text
    )
    # 2) Normalizar espacios
    text = re.sub(r"\s+", " ", text).strip()

    return text

In [25]:
def parse_ley_and_type(text: str):
    type_map = {
        "orgánica": "orgánica",
        "organica": "orgánica",
        "ordinaria": "ordinaria",
        "estatutaria": "estatutaria",
        "estaturaria": "estatutaria",
    }

    # Primero, intenta el patrón MÁS COMÚN:  "LEY ... No ... <número>"
    pattern_normal = r"""
        (?:LEY|LEYES)[\s\n]*          # "LEY" o "LEYES"
        (?:                             # Tipo de ley (opcional)
            (ESTATUTARIA|ORDINARIA|ORG[AÁ]NICA)
            [\s\n]+
        )?
        (?:N[°ºc]?\.?|No\.?)?[\s\n]*  # "N", "No", "N°", etc. (opcional)
        (\d+)                        # CAPTURA el número de ley
        (?:['´`‘’])?                 # Caracteres especiales opcionales
    """
    match = re.search(pattern_normal, text, re.IGNORECASE | re.VERBOSE | re.DOTALL)

    if match:  # Si ENCUENTRA el patrón normal, úsalo
        raw_type = match.group(1)
        ley_number = match.group(2)
        tipo_ley = type_map.get(raw_type.lower()) if raw_type else "ordinaria"
        return (tipo_ley, ley_number)


    # Si NO encuentra el patrón normal, intenta el patrón INVERSO: "<número> ... LEY"
    pattern_inverso = r"""
        (\d+)[\s\n]*          # CAPTURA el número de ley
        (?:LEY|LEYES)[\s\n]*
        (?:
            (ESTATUTARIA|ORDINARIA|ORG[AÁ]NICA)
            [\s\n]*
        )?
        (?:N[°ºc]?\.?|No\.?)?[\s\n]*

    """
    match = re.search(pattern_inverso, text, re.IGNORECASE | re.VERBOSE | re.DOTALL)

    if match:
      ley_number = match.group(1)
      raw_type = match.group(2)
      tipo_ley = type_map.get(raw_type.lower()) if raw_type else "ordinaria"
      return (tipo_ley, ley_number)


    return (None, None)

In [26]:
def parse_date_str(text: str) -> Optional[str]:
    patterns = [
        # 1) dd MMM yyyy (con espacios)
        r"(\d{1,2})\s+([A-Z]{3,4})\s+(\d{4})",
        # 2) ddMMMyyyy (sin espacios)
        r"(\d{1,2})([A-Z]{3,4})(\d{4})",
        # 3) Por si quedara algo como "dd-MM-aaaa"
        r"(\d{1,2})-(\b[A-Z]{3,4})-(\d{4})",
        # etc. Ajusta según tu realidad
    ]
    # 2) Mapeo de meses
    month_map = {
        "ENE": "01", "FEB": "02", "MAR": "03", "ABR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AGO": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DIC": "12"
    }
    all_matches = []  # lista de (start_index, dd, MMM, yyyy)
    for pattern in patterns:
        for m in re.finditer(pattern, text, flags=re.IGNORECASE):
            # Ejemplo: (day, month_str, year) => normalizar
            # OJO: depende del orden en el regex. Aquí asumo (1=day, 2=MMM, 3=year)
            start_idx = m.start()
            day = m.group(1)
            month_str = m.group(2).upper()
            year = m.group(3)
            all_matches.append((start_idx, day, month_str, year))
    if not all_matches:
        return None
    # 5) Ordenar por start_idx para que la última aparezca al final
    #    O más sencillo: quedarnos con max(all_matches, key=lambda x: x[0])
    #    y parsearlo
    last_match = max(all_matches, key=lambda x: x[0])  # la que tenga mayor start_index

    _, day, month_str, year = last_match

    # 6) Convertir day, month_str, year => dd/mm/yyyy
    day = day.zfill(2)
    month = month_map.get(month_str)
    if not month:
        return None  # mes desconocido
    return f"{day}/{month}/{year}"

In [27]:
def extraer_epigrafe(text: str, fecha_normalizada: str) -> str | None:
    if fecha_normalizada:
        try:
            datetime.strptime(fecha_normalizada, "%d/%m/%Y")
            start_index = text.find(fecha_normalizada)
            if start_index == -1:
                start_index = 0
                pattern_fecha = r"(\d{1,2})\s*([A-Za-z]{3,})\s*(\d{4})"
                match = re.search(pattern_fecha,text)
                if match:
                  start_index = match.end()
            else:
                start_index += len(fecha_normalizada)
        except ValueError:
           start_index = 0
           pattern_fecha = r"(\d{1,2})\s*([A-Za-z]{3,})\s*(\d{4})"
           match = re.search(pattern_fecha,text)
           if match:
              start_index = match.end()
    else:
        ley_match = re.search(r"(?:LEY|LEYES)[\s\n]*(?:N[°ºc]?\.?|No\.?)?[\s\n]*\d+", text, re.IGNORECASE)
        if ley_match:
            start_index = ley_match.end()
            mayuscula_match = re.search(r"[A-Z]", text[start_index:])
            if mayuscula_match:
                start_index += mayuscula_match.start()
        else:
            start_index = 0

    # Intenta buscar "EL CONGRESO" (con variaciones) o "REPUBLICA DE COLOMBIA"
    congreso_match = re.search(r"(?:EL|El|el)\s+CONGRESO|REPUBLICA\s+DE\s+COLOMBIA\s*-\s*GOBIERNO\s+NACIONAL", text, re.IGNORECASE)
    if congreso_match:
        end_index = congreso_match.start()
    else:
        end_index = len(text)

    if start_index < end_index:
        epigrafe = text[start_index:end_index].strip()
        # Priorizar la frase "POR"
        pattern_epigrafe = r"(POR\s+(?:MEDIO\s+DE\s+LA\s+CUAL|LA\s+CUAL)\s+.*?)(?:[\"“”\\s]*\n[\"“”\\s]*(?:EL|El|el)\s+CONGRESO|[\"“”\\s]*\n[\"“”\\s]*REPUBLICA\s+DE\s+COLOMBIA)"
        match_ep = re.search(pattern_epigrafe, epigrafe, re.IGNORECASE | re.DOTALL)
        if match_ep:  #Si encuentra un match con la frase
            return match_ep.group(1).strip()
        else:
          return epigrafe
    return None

In [28]:
def extract_first_section(text: str) -> str:
    """
    Devuelve el contenido desde el inicio del texto
    hasta la frase 'EL CONGRESO...' o 'EL CONGRESO DE LA REPUBLICA...'.
    Si no la encuentra, devuelve el texto completo.
    """
    pattern = r"^(.*?)(?=EL\s+CONGRESO\s+(?:DE\s+LA\s+REPUBLICA|DE\s+COLOMBIA))"
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1)
    return text

In [29]:
def extract_final_date(text: str) -> str | None:
    """
    Extrae la fecha de la sección final del documento ("Dada, a los...").
    Devuelve la fecha normalizada (DD/MM/YYYY) o None si no la encuentra.
    """
    # Modificación: Ahora solo busca la fecha
    pattern = r"Dada,?\s+a\s+los\s+(.+)"  # Busca "Dada, a los" y captura lo que sigue
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        date_str = match.group(1).strip()  # Obtiene la parte de la fecha
        return parse_date_str(date_str)  # Usa la función de parseo para normalizar
    return None

In [30]:
def extract_metadata_from_txt(txt_path: str) -> dict:
    """
    Lee un archivo .txt, extrae los metadatos y los devuelve como un diccionario.
    """
    with open(txt_path, "r", encoding="utf-8") as f:
        full_text = f.read()

    # Limpieza inicial
    full_text = fix_ocr_issues(full_text)

    # 1. Extraer de la sección inicial
    top_section = extract_first_section(full_text)
    metadata = {
        "numero_ley": None,
        "fecha": None,
        "tipo_ley": None,
        "epigrafe": None,
    }
    tipo_ley, ley_number = parse_ley_and_type(top_section)
    fecha_normalizada = parse_date_str(top_section)
    epigrafe = extraer_epigrafe(top_section, fecha_normalizada)

    metadata["numero_ley"] = ley_number
    metadata["tipo_ley"] = tipo_ley
    metadata["fecha"] = fecha_normalizada
    metadata["epigrafe"] = epigrafe

    # 2. Si NO hay fecha en la sección inicial, buscar en la sección final
    if not fecha_normalizada:
        fecha_normalizada = extract_final_date(full_text)  # Llama a la función modificada
        metadata["fecha"] = fecha_normalizada #Se actualiza

    # 3. Si NO hay número de ley, intentar extraer del texto COMPLETO
    if not metadata["numero_ley"]:
        tipo_ley_full, ley_number_full = parse_ley_and_type(full_text)  # Texto completo
        if ley_number_full:  #Si en el texto completo se encuentra el numero de ley.
            metadata["numero_ley"] = ley_number_full
        if tipo_ley_full:  #Si se encuentra el tipo de ley
            metadata["tipo_ley"] = tipo_ley_full


    return metadata

In [ ]:

# --- Funciones para procesar carpetas ---

def procesar_txt_en_carpeta(carpeta_txt, carpeta_salida):
    """
    Procesa todos los archivos .txt en una carpeta y guarda los metadatos
    en archivos JSON separados.
    """
    os.makedirs(carpeta_salida, exist_ok=True)  # Crea la carpeta de salida si no existe

    for archivo in os.listdir(carpeta_txt):
        if archivo.endswith(".txt"):
            ruta_txt = os.path.join(carpeta_txt, archivo)
            metadatos = extract_metadata_from_txt(ruta_txt)  # Usa la función para .txt

            nombre_json = os.path.splitext(archivo)[0] + ".json"
            ruta_json = os.path.join(carpeta_salida, nombre_json)

            with open(ruta_json, "w", encoding="utf-8") as f:
                json.dump(metadatos, f, ensure_ascii=False, indent=4)
            print(f"Metadatos guardados en: {ruta_json}")


# --- Ejemplo de Uso ---

# Ruta de la carpeta que contiene los archivos .txt
carpeta_txt = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\leyes"

# Ruta de la carpeta donde se guardarán los archivos JSON
carpeta_salida = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022" 

# Procesar todos los archivos .txt en la carpeta
procesar_txt_en_carpeta(carpeta_txt, carpeta_salida)

Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2182-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2183-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2184-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2185-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2186-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2187-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2188-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\OneDrive\Documents\proyect\document\json_output_2022\LEY-2189-2022plaintext.json
Metadatos guardados en: C:\Users\Jorge\O